# Optimizing with Tensorboard

The most popular use of Tensorboard is to manage multiple models. When training there are a lot of fine tuning to do that it gets tedious. Some parameters like the structure of the neural network, the optimizer used, the learning rate in the optimizer, etc. permutates to thousands to millions of possible models.



The things we are going to tweak in:

```model.add(Conv2D(64, (3,3), input_shape(X.shape[1:]))```

are the kernel size ```(3,3)```, number of layers, number of nodes per layer ```64```, and the choice of having a dense layer at the end.

Here we are permutating to all the possibilities based on the options above.

Now let's see it come together.

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

In [2]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
pickle_in = open("X_rgb.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y_rgb.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

In [4]:
dense_layers = [0, 1, 2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

In [ ]:
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
            
            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer - 1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

            for l in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))
            
            model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
            
            model.fit(X, y, batch_size=32, epochs=10, validation_split=0.3, callbacks=[tensorboard])

Train on 17462 samples, validate on 7484 samples
Epoch 1/10
17462/17462 [==============================] - 16s 943us/step - loss: 0.6261 - acc: 0.6643 - val_loss: 0.5547 - val_acc: 0.7217
Epoch 2/10
17462/17462 [==============================] - 13s 765us/step - loss: 0.5117 - acc: 0.7517 - val_loss: 0.5267 - val_acc: 0.7432
Epoch 3/10
17462/17462 [==============================] - 13s 771us/step - loss: 0.4594 - acc: 0.7843 - val_loss: 0.5264 - val_acc: 0.7410
Epoch 4/10
17462/17462 [==============================] - 13s 769us/step - loss: 0.4094 - acc: 0.8137 - val_loss: 0.5504 - val_acc: 0.7342
Epoch 5/10
17462/17462 [==============================] - 13s 767us/step - loss: 0.3605 - acc: 0.8426 - val_loss: 0.5587 - val_acc: 0.7338
Epoch 6/10
17462/17462 [==============================] - 13s 771us/step - loss: 0.3123 - acc: 0.8677 - val_loss: 0.5713 - val_acc: 0.7423
Epoch 7/10
17462/17462 [==============================] - 14s 777us/step - loss: 0.2684 - acc: 0.8900 - val_loss: 0.5